# Pareto Conditioned networks

Using the modified dam environment (less variance in the inflow), test PCN. This modifications is necessary because PCN expects deterministic transitions.

Also a modification to the original PCN code in morl-baselines is necessary in order to support the continuous actions of the environment:
- continuous_actions is a parameter given to the algorithm
- During training, actions are no longer sampled from a categorical distribution as in the [original paper](https://www.ifaamas.org/Proceedings/aamas2022/pdfs/p1110.pdf). This might hinder exploration...
- The logsoftmax function of the original model is no longer used, instead its output is a single value corresponding to an action
- The loss used in the model is now MSE instead of CE (regression loss vs classification)

In [5]:
import mo_gymnasium as mo_gym
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from morl_baselines.multi_policy.pcn.pcn import PCN

"""
Helper function to plot pf
"""
# TODO: add no objectives as parameter (later)
def plot_pf(file):
    columns = ["objective_1", "objective_2"]
    df = pd.read_csv(file, usecols=columns)

    plt.plot(df.objective_1, df.objective_2, 'o')
    plt.xlabel('Cost due to excess water level wrt flooding threshold upstream')
    plt.ylabel('Deficit in water supply wrt demand')
    plt.show()

## First attempt

- Dam inflow: mean = 40.0 and stdev = 1.0 
- Actions capped between [0.0;250.0]
- Scaling factor for desired horizon/return = [0.1, 0.1]
- Hypervolume ref point = [-100, -1000]

In [6]:
env = mo_gym.make('water-reservoir-v0', normalized_action=False, nO=2, penalize=True, time_limit=100)

PCNAgent = PCN(env, np.array((0.1, 0.1, 0.1), dtype=np.float32), continuous_actions=True)

PCNAgent.train(100000, env, np.array((-100.0,-1000), dtype=np.float32))

wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /tmp/PCN


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.